# General Race Positions Chart

## 1. Introduction

One file plot that's often seen as a very basic way to explain what happened in the race is the race history chart.
On the x-axis, this plot has the starting grid followed by all the laps. On the y-axis, you have the position. Individual drivers are plotted in the teamcolor where teammates are distinguished by the linestyle.<br>
It gives an idea about which drivers dominated the race and maybe stayed in the lead the whole race or which drivers recovered a lot of positions after a bad qualifying. Or likewise which drivers didn't have a very good race and dropped back.<br>
It does however only give you the positions of each car at each lap in the race. It doesn't say anything about the gaps or what the reason for the position change was. To explore that in more detail, we would need a different plot such as the Race History Chart. We'll discuss how to create them in a different notebook.

In [2]:
import re

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [3]:
grid_pattern = r"^GRID (\d{1,2} ){5,}"     # Example: "GRID 77 44 5 33 16 20 8 27 7 4 23 11 18 55 26 99"
lap_pattern = r"^LAP \d{1,2}( \d{1,2})+$"  # Example: "LAP 1 77 44 5 33 16 20 8 27 7 4 23 11 18 55 26 99"

In [4]:
def get_starting_grid_order(line, positions, expected_cars_at_grid):
    line_split = line.split()
    grid_order_strings = line_split[1:] # Leave out the word GRID which is at index 0.
            
    # Sanity check to see if 20 cars present. If for some reason a car didn't start or was disqualified,... we might notice it.
    if len(grid_order_strings) != expected_cars_at_grid:
        print("Different number of cars than expected on the grid!")
    else:
        grid_order = [int(x) for x in grid_order_strings]
        positions = {driver_num: (pos+1) for (pos, driver_num) in enumerate(grid_order)}
        return grid_order, positions

In [10]:
def add_lap_positions(line, positions):
    line_split = line.split()
    race_order_strings = line_split[2:]
            
    race_order = [int(x) for x in race_order_strings]
    
    for position, driver_number in enumerate(race_order, start=1):  #Start = 1 because first position is 1 and not 0.
        positions[driver_number] = np.append(positions[driver_number], int(position))
        
    return positions

In [12]:
expected_cars_at_grid = 20
filename = "Race History Text File.txt"
positions = {}

with open(filename) as file:
    for line in file:
        line = line.strip()
        
        # Look for the line with the Grid order in it. It always starts with GRID followed by the driver numbers.
        # Still check to see what happens with cars starting from the pitlane.
        if re.search(grid_pattern, line):
            grid_order, positions = get_starting_grid_order(line, positions, expected_cars_at_grid)
        # If the line, starts with Lap, we can get (the lap number and) the order of the cars on that lap.
        elif re.search(lap_pattern, line):
            add_lap_positions(line, positions)
            
race_history = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in positions.items() ]))

In [14]:
race_history

(59, 20)